In [1]:
import pandas as pd

df_pd = pd.read_csv("../Data/csv/reuters_all.csv")
# df_pd = df_pd.head(10)
print("📊 Tổng số bài viết:", len(df_pd))
print("📌 Các cột:", df_pd.columns.tolist())
print("\n📝 Xem Thông tin:")
print(df_pd.info())
df_pd = df_pd.dropna(subset=['body'])
df_pd = df_pd.reset_index(drop=True)
print(df_pd.info())

📊 Tổng số bài viết: 21578
📌 Các cột: ['id', 'title', 'body', 'topics']

📝 Xem Thông tin:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21578 entries, 0 to 21577
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      21578 non-null  int64 
 1   title   20841 non-null  object
 2   body    19043 non-null  object
 3   topics  11367 non-null  object
dtypes: int64(1), object(3)
memory usage: 674.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19043 entries, 0 to 19042
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19043 non-null  int64 
 1   title   19043 non-null  object
 2   body    19043 non-null  object
 3   topics  10377 non-null  object
dtypes: int64(1), object(3)
memory usage: 595.2+ KB
None


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws
import os

os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SemanticsBasedDistributedDocumentClustering_SparkNLP") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
    .config("spark.executorEnv.NLTK_DATA", "/Users/tuankietnguyen/nltk_data") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
df = spark.createDataFrame(df_pd)

df = df.withColumn("body", concat_ws(" ", col("title"), col("body")))
df.show(5)

25/07/04 12:36:16 WARN Utils: Your hostname, MBP-of-Yeun.local resolves to a loopback address: 127.0.0.1; using 192.168.50.97 instead (on interface en0)
25/07/04 12:36:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/tuankietnguyen/.ivy2/cache
The jars for the packages stored in: /Users/tuankietnguyen/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d2057d38-fc9e-460b-92f3-99f3176147e2;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in ce

+---+--------------------+--------------------+--------------------+
| id|               title|                body|              topics|
+---+--------------------+--------------------+--------------------+
|  1|  BAHIA COCOA REVIEW|BAHIA COCOA REVIE...|               cocoa|
|  2|STANDARD OIL <SRD...|STANDARD OIL <SRD...|                 NaN|
|  3|TEXAS COMMERCE BA...|TEXAS COMMERCE BA...|                 NaN|
|  4|TALKING POINT/BAN...|TALKING POINT/BAN...|                 NaN|
|  5|NATIONAL AVERAGE ...|NATIONAL AVERAGE ...|grain;wheat;corn;...|
+---+--------------------+--------------------+--------------------+
only showing top 5 rows



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63672)
Traceback (most recent call last):
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/tuankietnguyen/.pyenv/versions/3.10.16/li

In [ ]:
from pyspark.ml import Pipeline
from sparknlp.annotator import Tokenizer, Normalizer, StopWordsCleaner, LemmatizerModel
from sparknlp.base import DocumentAssembler, Finisher

document_assembler = DocumentAssembler().setInputCol("body").setOutputCol("document")

tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("raw_tokens")

finisher = Finisher() \
    .setInputCols(["raw_tokens"]) \
    .setOutputCols(["final_tokens"])

nlp_pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    finisher
])

print("⏳ Bắt đầu tiền xử lý với Spark NLP...")
df_processed = nlp_pipeline.fit(df).transform(df)
df_processed.cache()
print("✅ Xong bước NLP")

df_processed.select("body","final_tokens").show(5, truncate=False)

⏳ Bắt đầu tiền xử lý với Spark NLP...
✅ Xong bước NLP


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

25/07/04 13:58:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:611)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:610)
	at org.apache.spar

In [4]:
from pyspark.ml.clustering import KMeans, BisectingKMeans
from pyspark.sql.functions import col, split, explode, log2, countDistinct
def calculate_purity_entropy(df, label_col="topic", pred_col="prediction"):
    df_unique = df.select("id", pred_col, label_col).dropDuplicates(["id"])
    total = df_unique.select("id").distinct().count()
    if total == 0:
        return None, None
    counts = df.groupBy(pred_col, label_col).count()
    cluster_sizes = df.select(pred_col, "id").dropDuplicates(["id", pred_col]) \
    .groupBy(pred_col) \
    .agg(countDistinct("id").alias("cluster_total"))
    joined = counts.join(cluster_sizes, pred_col)

    max_counts = joined.groupBy(pred_col).agg({"count": "max"}).withColumnRenamed("max(count)", "max_count")
    total_purity = max_counts.agg({"max_count": "sum"}).collect()[0][0]
    purity = total_purity / total

    entropy_df = joined.withColumn("p", col("count") / col("cluster_total"))
    entropy_df = entropy_df.withColumn("p_log_p", -col("p") * log2(col("p")))
    entropy = entropy_df.groupBy(pred_col).agg({"p_log_p": "sum"}).agg({"sum(p_log_p)": "avg"}).collect()[0][0]

    return purity, entropy

results = []

variant = "Baseline"
from pyspark.ml.feature import IDF, HashingTF
numFeatures_stage = [9820]
for i in numFeatures_stage:
    print(f"\n============================== numFeatures={i} ==============================")
    hashingTF = HashingTF(inputCol="final_tokens", outputCol="rawFeatures", numFeatures=i)
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    df_tf = hashingTF.transform(df_processed)
    idf_model = idf.fit(df_tf)
    df_features = idf_model.transform(df_tf)
    df_features = df_features.select("id", "topics", "features")
    print(df_features.columns)
    # df_features.select("features").show(5, truncate=False)


    k_stage = [5,6,7,8]
    for k in k_stage:
        print(f"\n============================== k={k} ==============================")
        
        print(f"\nKMeans")
        km = KMeans(k=k, featuresCol="features", predictionCol="prediction", seed=42)
        pred_km = km.fit(df_features).transform(df_features)
        
        pred_km_filtered = pred_km.filter(col("topics").isNotNull() & (col("topics") != "NaN"))
        pred_km_filtered = pred_km_filtered.withColumn("topics_array", split(col("topics"), ";"))
        pred_km_exploded = pred_km_filtered.withColumn("topic", explode(col("topics_array")))
        
        purity_km, entropy_km = calculate_purity_entropy(pred_km_exploded)
        if purity_km is not None:
            print(f"Purity: {purity_km:.4f}")
            print(f"Entropy: {entropy_km:.4f}")
            results.append({
                "k": k,
                "method": "KMeans",
                "variant": variant,
                "purity": purity_km,
                "entropy": entropy_km
            })

        print(f"\nBisectingKMeans")
        bkm = BisectingKMeans(k=k, featuresCol="features", predictionCol="prediction", seed=42)
        pred_bkm = bkm.fit(df_features).transform(df_features)
        
        pred_bkm_filtered = pred_bkm.filter(col("topics").isNotNull() & (col("topics") != "NaN"))
        pred_bkm_filtered = pred_bkm_filtered.withColumn("topics_array", split(col("topics"), ";"))
        pred_bkm_exploded = pred_bkm_filtered.withColumn("topic", explode(col("topics_array")))
        
        purity_bkm, entropy_bkm = calculate_purity_entropy(pred_bkm_exploded)
        if purity_bkm is not None:
            print(f"Purity: {purity_bkm:.4f}")
            print(f"Entropy: {entropy_bkm:.4f}")
            results.append({
                "k": k,
                "method": "BisectingKMeans",
                "variant": variant,
                "purity": purity_bkm,
                "entropy": entropy_bkm
            })
spark.stop()
df_results = pd.DataFrame(results)
df_results.to_csv("../Data/csv/results.csv", index=False)
print("✅ Đã lưu kết quả vào 'results.csv'")


============================== numFeatures=9820 ==============================


['id', 'topics', 'features']

============================== k=5 ==============================

KMeans


OpenJDK 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
OpenJDK 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=39043Kb max_used=39052Kb free=92028Kb
 bounds [0x000000010a984000, 0x000000010cfe4000, 0x0000000112984000]
 total_blobs=14764 nmethods=13607 adapters=1070
 compilation: disabled (not enough contiguous free space left)


Purity: 0.4122
Entropy: 4.7739

BisectingKMeans


...

Purity: 0.4269
Entropy: 5.2476

============================== k=7 ==============================

KMeans


Purity: 0.4562
Entropy: 3.8089

BisectingKMeans


Purity: 0.4380
Entropy: 5.2335

============================== k=8 ==============================

KMeans


Purity: 0.4135
Entropy: 3.3794

BisectingKMeans


Purity: 0.4404
Entropy: 5.1710
✅ Đã lưu kết quả vào 'results.csv'


In [5]:
from nltk.corpus import wordnet as wn
def synset2concepts(sid, max_depth=2):
    from nltk.corpus import wordnet as wn
    try:
        syn = wn.synset(sid)
    except:
        return ["unknown"]
    
    paths = syn.hypernym_paths()
    if not paths:
        return [syn.lexname()]
    
    path = paths[0]
    concepts = []

    for i in range(0, min(len(path), max_depth + 1)):
        lemma = path[-(i + 1)].lemma_names()[0]
        if lemma not in concepts:
            concepts.append(lemma)
    return concepts
syn = wn.synset("cocoa.n.01")
print(syn.hypernym_paths())
print(synset2concepts("cocoa.n.01",2))

[[Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('matter.n.03'), Synset('substance.n.07'), Synset('food.n.01'), Synset('beverage.n.01'), Synset('cocoa.n.01')], [Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('matter.n.03'), Synset('substance.n.01'), Synset('fluid.n.01'), Synset('liquid.n.01'), Synset('beverage.n.01'), Synset('cocoa.n.01')], [Synset('entity.n.01'), Synset('abstraction.n.06'), Synset('relation.n.01'), Synset('part.n.01'), Synset('substance.n.01'), Synset('fluid.n.01'), Synset('liquid.n.01'), Synset('beverage.n.01'), Synset('cocoa.n.01')]]
['cocoa', 'beverage', 'food']


In [6]:
syn = wn.synset("review.n.02")
print(syn.lexname())

noun.communication
